In [2]:
import spacy
import torch
from transformers import CamembertTokenizer, CamembertForTokenClassification
from transformers import pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
import seaborn as sns
import tensorflow as tf
import sentencepiece

In [6]:
print("Vérification GPU...")
if tf.config.list_physical_devices('GPU'):
    print("GPU détecté")
else:
    print("Aucun GPU détecté, l'entraînement se fera sur le CPU")

Vérification GPU...
Aucun GPU détecté, l'entraînement se fera sur le CPU


In [8]:
def load_ner_dataset_file():
    ner_data = pd.read_csv("../../../../students_bootstrap/corpus/ner_dataset.csv", encoding='latin1', delimiter=",")
    print("Loaded ner_dataset file\n")
    return ner_data
    
def load_bottins_file():
    bottins_data = pd.read_csv("../../../../students_bootstrap/bottins.csv", encoding="utf-8", delimiter=",", header=None)
    bottins_data.columns = ["Text", "Source"]
    print("Loaded bottins file \n")
    return bottins_data

ner_data = load_ner_dataset_file()
print("Premières lignes du dataset ner_dataset.csv :\n")
print(ner_data.head())

print("\n######################################################################\n")

bottins_data = load_bottins_file()
print("Premières lignes du dataset bottins.csv :\n")
print(bottins_data.head())

Loaded ner_dataset file

Premières lignes du dataset ner_dataset.csv :

    Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O

######################################################################

Loaded bottins file 

Premières lignes du dataset bottins.csv :

                                                Text           Source
0  <PER>Dufan et Clémendot</PER>, <ACT>pharmacien...   "Bottin1_1820"
1  <PER>Dufant (Victor)</PER>, <ACT>libraire</ACT...   "Bottin1_1820"
2  <PER>Dufay</PER>, <ACT>essayeur du commerce</A...   "Bottin1_1820"
3  <PER>Dulay</PER>, <ACT>chandronnier</ACT>, <LO...   "Bottin1_1820"
4  <PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...   "Bottin1_1820"


In [10]:
# Nettoyage des données `bottins.csv` pour séparer le texte et les annotations
def clean_bottins_data(data):
    sentences = []
    for entry in data["Text"]:
        print(f"Original entry: {entry}")  # phrase originale
        cleaned_sentence = entry.split(",")[0]
        print(f"Cleaned sentence: {cleaned_sentence}")  # phrase nettoyée
        sentences.append(cleaned_sentence)
    return sentences

bottins_sentences = clean_bottins_data(bottins_data)

# affichage des premières phrases nettoyées
print(f"Sentences prétraitées : {bottins_sentences[:5]}")

Original entry: <PER>Dufan et Clémendot</PER>, <ACT>pharmaciens</ACT>, <LOC>r. de la Chaussée-d&apos;Antin</LOC>, <CARDINAL>34</CARDINAL>. <TITRE>(Elig.)</TITRE> 449
Cleaned sentence: <PER>Dufan et Clémendot</PER>
Original entry: <PER>Dufant (Victor)</PER>, <ACT>libraire</ACT>, <LOC>r. du Gros-Che- net</LOC>, <CARDINAL>2</CARDINAL>. 392
Cleaned sentence: <PER>Dufant (Victor)</PER>
Original entry: <PER>Dufay</PER>, <ACT>essayeur du commerce</ACT>, <LOC>place Dau- phine</LOC>, <CARDINAL>5</CARDINAL>.         355
Cleaned sentence: <PER>Dufay</PER>
Original entry: <PER>Dulay</PER>, <ACT>chandronnier</ACT>, <LOC>r. du Pont- aux Choux</LOC>, <CARDINAL>15</CARDINAL>. 314
Cleaned sentence: <PER>Dulay</PER>
Original entry: <PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>, <LOC>r. du Faub.-S. Denis</LOC>, <CARDINAL>20</CARDINAL>. 372
Cleaned sentence: <PER>Dufay (V.e)</PER>
Original entry: <PER>Dufay</PER>, <ACT>papetier</ACT>, <LOC>r. S.-Martin</LOC>, <CARDINAL>20</CARDINAL>. <CARDINAL>437</CARDIN